Prediction and Cleaning


In [6]:


import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import statsmodels.api as sm

# -----------------------------
# 1. Load and reshape the data
# -----------------------------
#DATA_PATH = "('/Users/sundeepravichander/OIM7502_F25/DC_housing_prices.csv') " # change if needed

df = pd.read_csv('/Users/sundeepravichander/OIM7502_F25/DC_housing_prices.csv')

# Identify the date columns (Zillow format → dates start around column 9)
date_cols = df.columns[9:]

# Reshape from wide → long
long_df = df.melt(
    id_vars=['RegionName'],     # ZIP code column
    value_vars=date_cols,       # all date columns
    var_name='date',            # new date column
    value_name='price'          # price values
)

# Clean & convert data types
long_df['zip'] = long_df['RegionName'].astype(str)
long_df['date'] = pd.to_datetime(long_df['date'])

# Keep only what you need
long_df = long_df[['zip', 'date', 'price']].sort_values(['zip', 'date']).reset_index(drop=True)

# Create simple time-series features
long_df['t'] = long_df.groupby('zip').cumcount()               # time index
long_df['month'] = long_df['date'].dt.month                    # month (for seasonality)

print(long_df.head())
print("\nData shape:", long_df.shape)

train_end = pd.to_datetime("2020-12-31")
val_end   = pd.to_datetime("2023-12-31")

def split_by_time(zip_df, train_end, val_end):
    """Split a single-zip dataframe into train / val / test by date."""
    train = zip_df[zip_df['date'] <= train_end].copy()
    val   = zip_df[(zip_df['date'] > train_end) & (zip_df['date'] <= val_end)].copy()
    test  = zip_df[zip_df['date'] > val_end].copy()
    return train, val, test

# -----------------------------
# 3. Helpers to build X, y
# -----------------------------
def build_regression_X(df_part, degree=1):
    """
    Build design matrix:
      - polynomial trend in t (degree 1 or 2)
      - month dummies for seasonality
    """
    df_part = df_part.reset_index(drop=True)
    X = pd.DataFrame({'t': df_part['t'].values})
    if degree >= 2:
        X['t2'] = df_part['t'].values ** 2

    # add month dummies (drop_first to avoid multicollinearity)
    month_dummies = pd.get_dummies(df_part['month'], prefix='month', drop_first=True)
    X = pd.concat([X, month_dummies], axis=1)
    return X

def evaluate_regression(train, val, degree):
    """Fit regression model of given degree and compute validation metrics."""
    if len(train) == 0 or len(val) == 0:
        return None, np.nan, np.nan

    X_train = build_regression_X(train, degree=degree)
    X_val   = build_regression_X(val, degree=degree)

    # Align columns in case some months are missing in val or train
    X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

    y_train = train['price'].values
    y_val   = val['price'].values

    model = LinearRegression()
    model.fit(X_train, y_train)

    val_pred = model.predict(X_val)

    rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    mape = mean_absolute_percentage_error(y_val, val_pred)

    return model, rmse, mape

def evaluate_naive(train, val):
    """Naive baseline: forecast = last observed value."""
    if len(train) == 0 or len(val) == 0:
        return np.nan, np.nan

    last_train_value = train['price'].iloc[-1]
    y_val = val['price'].values
    y_pred = np.full_like(y_val, fill_value=last_train_value, dtype=float)

    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mape = mean_absolute_percentage_error(y_val, y_pred)
    return rmse, mape

# -----------------------------
# 4. Loop over zips and evaluate models
# -----------------------------
results = []

for z in long_df['zip'].unique():
    z_df = long_df[long_df['zip'] == z].copy().sort_values('date')
    train, val, test = split_by_time(z_df, train_end, val_end)

    # Baseline naive
    naive_rmse, naive_mape = evaluate_naive(train, val)

    # Linear trend + seasonality (degree=1)
    lin_model, lin_rmse, lin_mape = evaluate_regression(train, val, degree=1)

    # Polynomial trend + seasonality (degree=2)
    poly_model, poly_rmse, poly_mape = evaluate_regression(train, val, degree=2)

    results.append({
        'zip': z,
        'naive_rmse': naive_rmse,
        'naive_mape': naive_mape,
        'lin_rmse': lin_rmse,
        'lin_mape': lin_mape,
        'poly_rmse': poly_rmse,
        'poly_mape': poly_mape
    })

metrics_df = pd.DataFrame(results)
print("\nValidation metrics by ZIP:")
print(metrics_df.sort_values('poly_mape').head())

# -----------------------------
# 5. Choose best regression model per zip
# -----------------------------
def choose_best_model(row):
    # choose lower MAPE between linear and polynomial
    if row['poly_mape'] < row['lin_mape']:
        return 'poly'
    else:
        return 'linear'

metrics_df['best_model'] = metrics_df.apply(choose_best_model, axis=1)
print("\nModel choice per ZIP:")
print(metrics_df[['zip', 'best_model', 'lin_mape', 'poly_mape']])

# -----------------------------
# 6. Refit best model on full history and forecast 2026–2027
# -----------------------------
future_dates = pd.date_range(start="2026-01-31", end="2027-12-31", freq="M")
forecast_rows = []

for _, row in metrics_df.iterrows():
    z = row['zip']
    best = row['best_model']

    z_df = long_df[long_df['zip'] == z].copy().sort_values('date')
    # recompute t for safety
    z_df['t'] = np.arange(len(z_df))
    z_df['month'] = z_df['date'].dt.month

    # fit on all available history up to last date
    degree = 2 if best == 'poly' else 1
    X_full = build_regression_X(z_df, degree=degree)
    y_full = z_df['price'].values

    model = LinearRegression()
    model.fit(X_full, y_full)

    # build future features
    last_t = z_df['t'].iloc[-1]
    future_t = np.arange(last_t + 1, last_t + 1 + len(future_dates))
    future_month = future_dates.month

    future_df = pd.DataFrame({
        't': future_t,
        'month': future_month
    })

    X_future = build_regression_X(future_df, degree=degree)
    # align cols with training X
    X_future = X_future.reindex(columns=X_full.columns, fill_value=0)

    future_pred = model.predict(X_future)

    for d, p in zip(future_dates, future_pred):
        forecast_rows.append({
            'zip': z,
            'date': d,
            'predicted_price': p,
            'model_type': f'{best}_trend_season'
        })

forecast_df = pd.DataFrame(forecast_rows)
print("\nSample of forecasted prices:")
print(forecast_df.head())

# Save to CSV: this is your final deliverable table
forecast_df.to_csv("DC_housing_forecast_2026_2027_by_zip.csv", index=False)
print("\nSaved forecasts to DC_housing_forecast_2026_2027_by_zip.csv")

     zip       date        price  t  month
0  20001 2000-01-31  152793.9122  0      1
1  20001 2000-02-29  153157.2965  1      2
2  20001 2000-03-31  153360.9641  2      3
3  20001 2000-04-30  154323.7478  3      4
4  20001 2000-05-31  155195.4995  4      5

Data shape: (6489, 5)

Validation metrics by ZIP:
     zip    naive_rmse  naive_mape      lin_rmse  lin_mape     poly_rmse  \
4  20005  10905.056184    0.017668  84812.022007  0.155115  16147.692699   
8  20010  34910.494835    0.038441  31009.214776  0.036749  31388.463054   
7  20009   8629.729607    0.012138  54257.849487  0.085505  24739.360856   
9  20011  38861.730842    0.049998  68903.688391  0.091616  41011.596214   
1  20002  18850.747820    0.023239  35517.959205  0.047615  51354.703717   

   poly_mape  
4   0.028626  
8   0.037287  
7   0.038854  
9   0.052305  
1   0.054390  

Model choice per ZIP:
      zip best_model  lin_mape  poly_mape
0   20001       poly  0.160743   0.081603
1   20002     linear  0.047615   0.05

/var/folders/2n/lq__dftn5yb18dj6bbxg48jw0000gn/T/ipykernel_1345/3389992662.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  long_df['date'] = pd.to_datetime(long_df['date'])
/var/folders/2n/lq__dftn5yb18dj6bbxg48jw0000gn/T/ipykernel_1345/3389992662.py:155: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start="2026-01-31", end="2027-12-31", freq="M")


Test multiple regression approaches (linear, polynomial, time-series models like ARIMA)

Validate model performance and select best-performing models

In [36]:

#Time-based train/validation split
train_end = pd.to_datetime("2020-12-31")
val_end   = pd.to_datetime("2023-12-31")


#Compute error metrics for each model
def evaluate_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    return rmse, mape


# LINEAR REGRESSION
def run_linear_regression(train, val):
    X_train = pd.get_dummies(train[['t','month']], columns=['month'], drop_first=True)
    X_val   = pd.get_dummies(val[['t','month']],   columns=['month'], drop_first=True)
    X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

    y_train = train['price']
    y_val   = val['price']

    model = LinearRegression()
    model.fit(X_train, y_train)

    preds = model.predict(X_val)
    rmse, mape = evaluate_metrics(y_val, preds)


#POLYNOMIAL REGRESSION MODEL    

    return model, rmse, mape
def run_polynomial_regression(train, val):
    train = train.copy()
    val   = val.copy()

    train['t2'] = train['t']**2
    val['t2']   = val['t']**2

    X_train = pd.get_dummies(train[['t','t2','month']], columns=['month'], drop_first=True)
    X_val   = pd.get_dummies(val[['t','t2','month']],   columns=['month'], drop_first=True)
    X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

    y_train = train['price']
    y_val   = val['price']

    model = LinearRegression()
    model.fit(X_train, y_train)

    preds = model.predict(X_val)
    rmse, mape = evaluate_metrics(y_val, preds)

    return model, rmse, mape

#  ARIMA TIME SERIES MODEL
    
import warnings
warnings.filterwarnings("ignore")

def run_arima(train, val, order=(1,1,0)):
    series_train = train.set_index('date')['price']
    series_val   = val.set_index('date')['price']

    # Force real monthly frequency (month-end "ME")
    series_train.index = pd.DatetimeIndex(series_train.index).to_period('M').to_timestamp('M')
    series_val.index   = pd.DatetimeIndex(series_val.index).to_period('M').to_timestamp('M')

    series_train = series_train.asfreq('ME')
    series_val   = series_val.asfreq('ME')

    # Fit ARIMA safely
    model = sm.tsa.ARIMA(series_train, order=order)
    fit = model.fit()

    preds = fit.forecast(steps=len(series_val))

    rmse, mape = evaluate_metrics(series_val.values, preds.values)
    return fit, rmse, mape

#Evaluation of 3 models
results = []

for z in long_df['zip'].unique():

    z_df = long_df[long_df['zip'] == z].copy()
    train = z_df[z_df['date'] <= train_end]
    val   = z_df[(z_df['date'] > train_end) & (z_df['date'] <= val_end)]

    # Model 1
    lin_model, lin_rmse, lin_mape = run_linear_regression(train, val)

    # Model 2
    poly_model, poly_rmse, poly_mape = run_polynomial_regression(train, val)

    # Model 3
    arima_model, arima_rmse, arima_mape = run_arima(train, val)
    
#Select best-performing models
    results.append({
        'zip': z,
        'linear_mape': lin_mape,
        'poly_mape': poly_mape,
        'arima_mape': arima_mape
    })

results_df = pd.DataFrame(results)
print(results_df)


def choose_model(row):
    mape_scores = {
        'linear': row['linear_mape'],
        'poly': row['poly_mape'],
        'arima': row['arima_mape']
    }
    return min(mape_scores, key=mape_scores.get)

results_df['best_model'] = results_df.apply(choose_model, axis=1)
print(results_df)



      zip  linear_mape  poly_mape  arima_mape
0   20001     0.160743   0.081603    0.020553
1   20002     0.047615   0.054390    0.023026
2   20003     0.033861   0.063689    0.032627
3   20004     0.202761   0.087586    0.016953
4   20005     0.155115   0.028626    0.017625
5   20007     0.038527   0.144770    0.076756
6   20008     0.028802   0.133201    0.056907
7   20009     0.085505   0.038854    0.012131
8   20010     0.036749   0.037287    0.037723
9   20011     0.091616   0.052305    0.048706
10  20012     0.110122   0.128339    0.068035
11  20015     0.115635   0.205738    0.115579
12  20016     0.097108   0.174053    0.094045
13  20017     0.090470   0.057478    0.033729
14  20018     0.117685   0.071605    0.059161
15  20019     0.198472   0.117002    0.038229
16  20020     0.187839   0.149584    0.047101
17  20024     0.051217   0.058662    0.026014
18  20032     0.246319   0.127456    0.048643
19  20036     0.134789   0.061521    0.023036
20  20037     0.112006   0.070653 

In [38]:
from sklearn.linear_model import LinearRegression

# 1) Define forecast period (month-end)
future_dates = pd.date_range(start="2026-01-31", end="2027-12-31", freq="ME")

forecast_rows = []

for _, row in results_df.iterrows():
    z = row['zip']
    best = row['best_model']

    # Get full history for this ZIP
    z_df = long_df[long_df['zip'] == z].copy().sort_values('date')

    # Rebuild time features
    z_df['t'] = np.arange(len(z_df))
    z_df['month'] = z_df['date'].dt.month

    # ---------------- LINEAR MODEL ----------------
    if best == 'linear':
        X_full = pd.get_dummies(z_df[['t', 'month']], columns=['month'], drop_first=True)
        y_full = z_df['price'].values

        model = LinearRegression()
        model.fit(X_full, y_full)

        # future t and month
        last_t = z_df['t'].iloc[-1]
        future_t = np.arange(last_t + 1, last_t + 1 + len(future_dates))
        future_months = future_dates.month

        future_df = pd.DataFrame({'t': future_t, 'month': future_months})
        X_future = pd.get_dummies(future_df, columns=['month'], drop_first=True)
        X_future = X_future.reindex(columns=X_full.columns, fill_value=0)

        preds = model.predict(X_future)

    # -------------- POLYNOMIAL MODEL --------------
    elif best == 'poly':
        z_df['t2'] = z_df['t']**2

        X_full = pd.get_dummies(z_df[['t', 't2', 'month']], columns=['month'], drop_first=True)
        y_full = z_df['price'].values

        model = LinearRegression()
        model.fit(X_full, y_full)

        last_t = z_df['t'].iloc[-1]
        future_t = np.arange(last_t + 1, last_t + 1 + len(future_dates))
        future_df = pd.DataFrame({
            't': future_t,
            't2': future_t**2,
            'month': future_dates.month
        })

        X_future = pd.get_dummies(future_df, columns=['month'], drop_first=True)
        X_future = X_future.reindex(columns=X_full.columns, fill_value=0)

        preds = model.predict(X_future)

    # ------------------ ARIMA MODEL ----------------
    elif best == 'arima':
        # price as a time series
        series = z_df.set_index('date')['price']

        # make sure index is proper month-end freq
        series.index = pd.DatetimeIndex(series.index).to_period('M').to_timestamp('M')
        series = series.asfreq('ME')

        # fit ARIMA with same order you used in run_arima
        model = sm.tsa.ARIMA(series, order=(1, 1, 0)).fit()

        preds = model.forecast(steps=len(future_dates))

    # collect forecasts for this ZIP
    for d, p in zip(future_dates, preds):
        forecast_rows.append({
            'zip': z,
            'date': d,
            'predicted_price': float(p),
            'model_used': best
        })

# 2) Final forecast table
forecast_df = pd.DataFrame(forecast_rows)
forecast_df = forecast_df.sort_values(['zip', 'date']).reset_index(drop=True)

print("\n====== SAMPLE OF ZIP-LEVEL FORECASTS (2026–2027) ======\n")
print(forecast_df.head(20).to_string(index=False))

# 3) Save to CSV for your team/report
forecast_df.to_csv("DC_ZIP_Forecasts_2026_2027.csv", index=False)
print("\nSaved: DC_ZIP_Forecasts_2026_2027.csv")


results_df.sort_values(["best_model", "linear_mape", "poly_mape", "arima_mape"])



====== SAMPLE OF ZIP-LEVEL FORECASTS (2026–2027) ======

  zip       date  predicted_price model_used
20001 2026-01-31    644986.511534      arima
20001 2026-02-28    644966.359111      arima
20001 2026-03-31    644962.908295      arima
20001 2026-04-30    644962.317392      arima
20001 2026-05-31    644962.216209      arima
20001 2026-06-30    644962.198882      arima
20001 2026-07-31    644962.195916      arima
20001 2026-08-31    644962.195408      arima
20001 2026-09-30    644962.195321      arima
20001 2026-10-31    644962.195306      arima
20001 2026-11-30    644962.195303      arima
20001 2026-12-31    644962.195303      arima
20001 2027-01-31    644962.195303      arima
20001 2027-02-28    644962.195303      arima
20001 2027-03-31    644962.195303      arima
20001 2027-04-30    644962.195303      arima
20001 2027-05-31    644962.195303      arima
20001 2027-06-30    644962.195303      arima
20001 2027-07-31    644962.195303      arima
20001 2027-08-31    644962.195303      ari

,zip,linear_mape,poly_mape,arima_mape,best_model
2,20003,0.033861,0.063689,0.032627,arima
1,20002,0.047615,0.054390,0.023026,arima
17,20024,0.051217,0.058662,0.026014,arima
7,20009,0.085505,0.038854,0.012131,arima
13,20017,0.090470,0.057478,0.033729,arima
9,20011,0.091616,0.052305,0.048706,arima
12,20016,0.097108,0.174053,0.094045,arima
10,20012,0.110122,0.128339,0.068035,arima
20,20037,0.112006,0.070653,0.014949,arima
11,20015,0.115635,0.205738,0.115579,arima


In [30]:
results = []

for z in long_df['zip'].unique():

    z_df = long_df[long_df['zip'] == z].copy()
    train = z_df[z_df['date'] <= train_end]
    val   = z_df[(z_df['date'] > train_end) & (z_df['date'] <= val_end)]

    # Model 1
    lin_model, lin_rmse, lin_mape = run_linear_regression(train, val)

    # Model 2
    poly_model, poly_rmse, poly_mape = run_polynomial_regression(train, val)

    # Model 3
    arima_model, arima_rmse, arima_mape = run_arima(train, val)

    results.append({
        'zip': z,
        'linear_rmse': lin_rmse,
        'linear_mape': lin_mape,
        'poly_rmse': poly_rmse,
        'poly_mape': poly_mape,
        'arima_rmse': arima_rmse,
        'arima_mape': arima_mape
    })

results_df = pd.DataFrame(results)
print(results_df)


      zip    linear_rmse  linear_mape      poly_rmse  poly_mape  \
0   20001  119051.013889     0.160743   63426.348032   0.081603   
1   20002   35517.959205     0.047615   51354.703717   0.054390   
2   20003   33743.172380     0.033861   60855.160964   0.063689   
3   20004   95289.243282     0.202761   41489.467416   0.087586   
4   20005   84812.022007     0.155115   16147.692699   0.028626   
5   20007   52137.875810     0.038527  175339.916250   0.144770   
6   20008   32113.329462     0.028802  113596.972981   0.133201   
7   20009   54257.849487     0.085505   24739.360856   0.038854   
8   20010   31009.214776     0.036749   31388.463054   0.037287   
9   20011   68903.688391     0.091616   41011.596214   0.052305   
10  20012   91279.810551     0.110122  105328.762212   0.128339   
11  20015  156429.627390     0.115635  274451.041298   0.205738   
12  20016  108151.442614     0.097108  190590.965904   0.174053   
13  20017   59843.853362     0.090470   39172.971355   0.05747

In order to keep the models clean and realistic we had to make a few assumptions. 
Main one being that we assume that the historical price patterns we see in Zillow like the  upward trends the monthly fluctuations would continue even during  2026 & 2027. Each ZIP code was treated as its own market, as not every neighborhood behaves in a similar manner.
For the regression model we assumed that the housing prices will change with time and seasonality, while in cotrast the ARIMA model assumes that they will become stabel when we get rid of the long term drift. 

At the same time, the models do have limitiations as does everything else. While forecasting for two years does the models cant  anticipate uncertainty and unexpected events like interest-rate changes or economic shocks or global geopolitical  situations that  can shift the market in ways the model cannot comprehend.Our models only use historical price data and don’t directly include outside factors such as mortgage rates, supply constraints, or policy changes, which can all affect housing markets. Polynomial models can sometimes over-interpret noise as curvature, and ARIMA can struggle to converge or capture long-term upward trends. We minimized these issues by validating everything on recent years rather than just trusting the fit on the full dataset.

In order to  measure  the true performance of each model,  RMSE(Root Mean Square Error) & MAPE (Mean Absolute Percentage Error) were used as metrics for the comparison. The RMSE tells us how far off the predictions were in $s. MAPE expresses the erroe as a percentage , thus making it easy to compare ZIP codes with different price levels in a more simpler manner. The dataset was trained on 3 models namely Linear, Polynomial and ARIMA. This used data through 2020 and validated the same across the 2021-2023 period.Overall, most ZIP codes showed average errors in the 3–7% range, which indicates solid predictive performance. After selecting the best model for each ZIP, we used it to generate month-by-month forecasts for 2026 and 2027. 

Based on the  results,we can conclude that ARIMA  achieved the lowest  (MAPE) for most of the ZIP codes and is the most suitable model. This indicates that DC housing prices exhibit strong month-to-month autocorrelation patterns, which ARIMA captures more effectively than linear or polynomial trend models. 